In [2]:
import json
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import clip
import torch
import csv
import base64
from tqdm import tqdm

all_dict = {}
dir_list = os.listdir('objects_list/')
for path in dir_list:
    with open('objects_list/'+path, 'r') as f:
        data = json.load(f)
        all_dict[path[0:11]] = data

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("RN50x4", device=device, download_root = "/home/wsco/DILLM/img_features/")

TSV_FIELDNAMES = ['scanId', 'viewpointId', 'features']
OUTFILE = '/home/wsco/DILLM/img_features/CLIP-obj-RN50-1e-6.tsv'
with open(OUTFILE, 'wt') as tsvfile:
    writer = csv.DictWriter(tsvfile, delimiter = '\t', fieldnames = TSV_FIELDNAMES)
    all_scan = list(all_dict.keys())
    for i in tqdm(range(len(all_scan))):
    # traversal scans
        a = all_dict[all_scan[i]]
        all_viewpoint = list(a.keys())
        for j in range(len(all_viewpoint)):
        # traversal viewponits
            obj_list = []
            valid_index = []
            for k in range(12):
                # traversal 12 directions
                if a[all_viewpoint[j]][k] != []:
                    obj = ''
                    for m in range(len(a[all_viewpoint[j]][k])):
                        # traversal objects
                        obj += a[all_viewpoint[j]][k][m]
                        obj += ' '
                    obj_list.append(obj)
                    valid_index.append(k)
                else:
                    pass
            # print(obj_list)
            text = clip.tokenize(obj_list, truncate=True).to(device) # [3, 77]
            with torch.no_grad():
                # avoid nan in softmax when there is no objects
                new_text_features = torch.ones((12,640)) * 1e-6
                text_features = model.encode_text(text)
                # print(text_features.shape)
                if valid_index != []:
                    for n in range(len(valid_index)):
                        new_text_features[valid_index[n]] = text_features[n]
                # print(new_text_features.shape)
                features = new_text_features.detach().numpy()
                # print(features.shape)
                writer.writerow({
                'scanId': all_scan[i],
                'viewpointId': all_viewpoint[j],
                'features': str(base64.b64encode(features), "utf-8")
                })

100%|██████████| 90/90 [01:27<00:00,  1.03it/s]
